In [1]:
import os
import pandas as pd
os.chdir('/home/reffert/DeepAR_InfluenzaForecast')
from PythonFiles.model import preprocessing, train_test_split
from PythonFiles.Configuration import Configuration
from PythonFiles.HpTuning import get_data, objectiveFNN
from ray.air import session
from ray import tune
from datetime import datetime
configuration = Configuration()
configuration = Configuration()
configuration.train_start_time = datetime(1999,1,1,0,0,0)
configuration.train_end_time = datetime(2018,9,30,23,0,0)
configuration.test_end_time = datetime(2020,9,30,23,0,0)
# import the data
influenza_df = pd.read_csv("/home/reffert/DeepAR_InfluenzaForecast/Notebooks/DataProcessing/influenza.csv", sep=',')
population_df = pd.read_csv("/home/reffert/DeepAR_InfluenzaForecast/Notebooks/DataProcessing/PopulationVector.csv", sep=',')
neighbourhood_df = pd.read_csv("/home/reffert/DeepAR_InfluenzaForecast/Notebooks/DataProcessing/AdjacentMatrix.csv", sep=',', index_col=0)

data_splits_dict = {}
locations = list(influenza_df.location.unique())
#Process the df into a uniformly spaced df
df = influenza_df.loc[influenza_df.location.isin(locations), ['value', 'location', 'date','week']]
df = preprocessing(configuration, df, check_count=False, output_type="corrected_df")
for location in locations:
    df.loc[df.location == location, "population"] = int(population_df.loc[population_df.Location == location, "2011"].values[0])
    df.loc[df.location == location, locations] = neighbourhood_df.loc[neighbourhood_df.index==location,locations].values[0].astype(int)
data_splits_dict[f"with_features_2001"] = list(train_test_split(configuration, df, True))
train = data_splits_dict["with_features_2001"][0]
test = data_splits_dict["with_features_2001"][1]

In [ ]:
hp_search_space = {
    "num_hidden_dimensions": tune.grid_search([None]),
    "context_length":tune.grid_search([None]),
    "epochs":tune.grid_search([100]),
    "prediction_length" : tune.grid_search([4]), 
    "num_batches_per_epoch":tune.grid_search([50]),
    "batch_normalization":tune.grid_search([False]),
    "batch_size":tune.grid_search([32]),
    }


tuner = tune.Tuner(
    tune.with_parameters(objectiveFNN, train=train, test=test, configuration=configuration),
    tune_config=tune.TuneConfig(
        num_samples = 100,
        metric="mean_WIS",
        mode="min",
        max_concurrent_trials=8,
    ),
    param_space=hp_search_space,
)
results = tuner.fit()

print("Best hyperparameters found were: ", results.get_best_result().config)

results_df = results.get_dataframe()
print(results_df)
results_df.to_csv("default_FNN_results_06_06.csv")

2023-06-06 12:04:48,730	INFO worker.py:1553 -- Started a local Ray instance.


  0%|          | 0/50 [00:00<?, ?it/s]m 
Running evaluation: 0it [00:00, ?it/s]m 
Running evaluation: 0it [00:00, ?it/s]m 
Running evaluation: 41511it [00:02, 18202.74it/s]
  0%|          | 0/50 [00:00<?, ?it/s]m 
Running evaluation: 41511it [00:03, 11669.56it/s]
  0%|          | 0/50 [00:00<?, ?it/s]m 
Running evaluation: 0it [00:00, ?it/s]m 
Running evaluation: 0it [00:00, ?it/s]m 
Running evaluation: 0it [00:00, ?it/s]m 
  0%|          | 0/50 [00:00<?, ?it/s]m 
Running evaluation: 41511it [00:02, 14225.26it/s]
  0%|          | 0/50 [00:00<?, ?it/s]m 
Running evaluation: 41511it [00:04, 9180.95it/s]
Running evaluation: 41511it [00:05, 8111.24it/s]
100%|██████████| 50/50 [00:10<00:00,  4.71it/s, epoch=100/100, avg_epoch_loss=0.921]
(objectiveFNN pid=3073875) /home/reffert/.conda/envs/TEST_ENV/lib/python3.10/site-packages/pandas/core/dtypes/cast.py:1181: UserWarning: Warning: converting a masked element to nan.
(objectiveFNN pid=3073875)   return arr.astype(dtype, copy=True)
(objective

Trial name,date,done,episodes_total,experiment_id,experiment_tag,hostname,iterations_since_restore,mean_WIS,node_ip,pid,time_since_restore,time_this_iter_s,time_total_s,timestamp,timesteps_since_restore,timesteps_total,training_iteration,trial_id,warmup_time
objectiveFNN_976dd_00000,2023-06-06_12-14-43,True,,ea38b56b78754d41916f8cb0125c819b,"0_batch_normalization=False,batch_size=32,context_length=None,epochs=100,num_batches_per_epoch=50,num_hidden_dimensions=None,prediction_length=4",econ-stat-rr01,1,429.134,172.22.1.197,3073740,579.54,579.54,579.54,1686046483,0,,1,976dd_00000,0.00575662
objectiveFNN_976dd_00001,2023-06-06_12-13-40,True,,b1c4d96dc819458e9487431cea29ed82,"1_batch_normalization=False,batch_size=32,context_length=None,epochs=100,num_batches_per_epoch=50,num_hidden_dimensions=None,prediction_length=4",econ-stat-rr01,1,415.691,172.22.1.197,3073869,511.021,511.021,511.021,1686046420,0,,1,976dd_00001,0.00655222
objectiveFNN_976dd_00002,2023-06-06_12-13-56,True,,26fc18d50ae34f878f5aefd9a15270d1,"2_batch_normalization=False,batch_size=32,context_length=None,epochs=100,num_batches_per_epoch=50,num_hidden_dimensions=None,prediction_length=4",econ-stat-rr01,1,414.174,172.22.1.197,3073871,526.987,526.987,526.987,1686046436,0,,1,976dd_00002,0.00721598
objectiveFNN_976dd_00003,2023-06-06_12-14-38,True,,6ae069f071884b8689e22a34ceef9257,"3_batch_normalization=False,batch_size=32,context_length=None,epochs=100,num_batches_per_epoch=50,num_hidden_dimensions=None,prediction_length=4",econ-stat-rr01,1,418.017,172.22.1.197,3073873,568.921,568.921,568.921,1686046478,0,,1,976dd_00003,0.00759172
objectiveFNN_976dd_00004,2023-06-06_12-13-42,True,,0b3d0f4096664b868dc4a8897dc03946,"4_batch_normalization=False,batch_size=32,context_length=None,epochs=100,num_batches_per_epoch=50,num_hidden_dimensions=None,prediction_length=4",econ-stat-rr01,1,441.987,172.22.1.197,3073875,512.838,512.838,512.838,1686046422,0,,1,976dd_00004,0.00671721
objectiveFNN_976dd_00005,2023-06-06_12-13-51,True,,20a3c6223d384283971c190f5b48d683,"5_batch_normalization=False,batch_size=32,context_length=None,epochs=100,num_batches_per_epoch=50,num_hidden_dimensions=None,prediction_length=4",econ-stat-rr01,1,446.918,172.22.1.197,3073877,521.94,521.94,521.94,1686046431,0,,1,976dd_00005,0.00706434
objectiveFNN_976dd_00006,2023-06-06_12-13-53,True,,d8459da46f98400d813c5f77c76ad56a,"6_batch_normalization=False,batch_size=32,context_length=None,epochs=100,num_batches_per_epoch=50,num_hidden_dimensions=None,prediction_length=4",econ-stat-rr01,1,433.764,172.22.1.197,3073879,523.678,523.678,523.678,1686046433,0,,1,976dd_00006,0.00582838
objectiveFNN_976dd_00007,2023-06-06_12-15-11,True,,2e24ff7c103c4671a29b178e821aabea,"7_batch_normalization=False,batch_size=32,context_length=None,epochs=100,num_batches_per_epoch=50,num_hidden_dimensions=None,prediction_length=4",econ-stat-rr01,1,426.507,172.22.1.197,3073881,601.768,601.768,601.768,1686046511,0,,1,976dd_00007,0.00542021
objectiveFNN_976dd_00008,2023-06-06_12-23-58,True,,b1c4d96dc819458e9487431cea29ed82,"8_batch_normalization=False,batch_size=32,context_length=None,epochs=100,num_batches_per_epoch=50,num_hidden_dimensions=None,prediction_length=4",econ-stat-rr01,1,421.434,172.22.1.197,3073869,617.502,617.502,617.502,1686047038,0,,1,976dd_00008,0.00655222
objectiveFNN_976dd_00009,2023-06-06_12-23-54,True,,0b3d0f4096664b868dc4a8897dc03946,"9_batch_normalization=False,batch_size=32,context_length=None,epochs=100,num_batches_per_epoch=50,num_hidden_dimensions=None,prediction_length=4",econ-stat-rr01,1,420.546,172.22.1.197,3073875,611.524,611.524,611.524,1686047034,0,,1,976dd_00009,0.00671721


  0%|          | 0/50 [00:00<?, ?it/s]m 
Running evaluation: 0it [00:00, ?it/s]m 
(objectiveFNN pid=3073877) /home/reffert/.conda/envs/TEST_ENV/lib/python3.10/site-packages/pandas/core/dtypes/cast.py:1181: UserWarning: Warning: converting a masked element to nan.
(objectiveFNN pid=3073877)   return arr.astype(dtype, copy=True)
  0%|          | 0/50 [00:00<?, ?it/s]m 
(objectiveFNN pid=3073879) /home/reffert/.conda/envs/TEST_ENV/lib/python3.10/site-packages/pandas/core/dtypes/cast.py:1181: UserWarning: Warning: converting a masked element to nan.
(objectiveFNN pid=3073879)   return arr.astype(dtype, copy=True)
  0%|          | 0/50 [00:00<?, ?it/s]m 
Running evaluation: 41511it [00:05, 7312.22it/s]
(objectiveFNN pid=3073871) /home/reffert/.conda/envs/TEST_ENV/lib/python3.10/site-packages/pandas/core/dtypes/cast.py:1181: UserWarning: Warning: converting a masked element to nan.
(objectiveFNN pid=3073871)   return arr.astype(dtype, copy=True)
  0%|          | 0/50 [00:00<?, ?it/s]m 
Runni